# Course Timetabling: Estudo de caso para o problema de agendamento de grade horária

## Fase 1: Alocação de professor em disciplina

Objetivo:
- Maximizar a performance acadêmica (EAP, professor habilitado em dar a disciplina) e o número de restrições fracas atendidas
- Alocar professores efetivos e substitutos em disciplinas (obrigatórias, eletivas e de serviço) conforme as regras do instituto da computação da UFRJ

### Modelagem

Conjuntos:

- $p \in P$: Conjunto de professsores
- $d \in D$: Conjunto de disciplinas ofertadas
- $Cred_d$: Conjunto de créditos de cada disciplina
- $D_p \subset D$: Preferências de disciplinas de cada professor p
- $B_p \subset D$: Disciplinas nas quais o professor p está habilitado a dar aulas.

<!-- - $Tp \subset H Preferência de turno de cada professor p; -->

#### Variáveis

1) Alocação de professor em disciplina

$$
X_{p,d} \in (0,1) =

\begin{cases}
1 &  \quad \text{se o professor p for alocado na disciplina d}\\ 
0 & \quad \text{caso contrário}
\end{cases}

$$

2) Professor habilitado em dar disciplina

$$

b_p \in (0,1) = 

\begin{cases}
1 &  \quad d \in B_p\\ 
0 & \quad \text{caso contrário}
\end{cases}

$$

3) Professor com preferência em dar uma determinada disciplina

$$

k_p \in (0,1) = 

\begin{cases}
1 &  \quad d \in D_p\\ 
0 & \quad \text{caso contrário}
\end{cases}

$$

#### Restrições

##### Restrições fortes

1) Regime de trabalho (quantidade de horas). Para cada equação, temos o professor p fixado e pertencente ao conjunto P.

- $cred_{min}$: Número de créditos mínimos: 8. Cada professor precisa dar no mínimo 8 créditos horas de aula por semana (cada disciplina geralmente possui 4 créditos).

$$
\sum_{d \in D} cred_d X_{p,d} \geq cred_{min}
$$


- $cred_{max}$: Número de créditos máximos: 12. Às vezes professores podem dar 12 créditos por semana.

$$
\sum_{d \in D} cred_d X_{p,d} \leq cred_{max}
$$








- $qtd\_{prof}$ : Número de professores a escolher: 1

2) Professores substitutos 

3) Habilitação em disciplinas
- Cada professor deverá dar pelo menos uma aula atrelada a área de conhecimento de seu concurso

2) Professores substitutos 

3) Habilitação em disciplinas
- Cada professor deverá dar pelo menos uma aula atrelada a área de conhecimento de seu concurso

##### Restrições fracas

OBS.: Válido somente para professor efetivo

1) Preferência do professor em dar uma disciplina
2) Rotatividade de professores: cada professor deverá permanecer dando uma mesma disciplina por no máximo X períodos (ex.: 3) #TODO


#### Função objetivo

$$
f_{p,d} = b_p 
$$